# Used cars analysis from eBay Kleinanzeigen
In this project, we'll work with a dataset of used cars from eBay Kleinanzeigen, a classified section of the German eBay website.

The aim of this project is to clean the data and analyze the included used car listings. The listing contains 50.000 data points from the original dataset.

### The Dataset.
The dataset was originally scraped and uploaded to [Kaggle](https://www.kaggle.com/orgesleka/used-cars-database/data). We sampled 50.000 data points from the full dataset of 371.824 in total.

**The data dictionary provided with data is as follows:**
* `dateCrawled` - When this ad was first crawled. All field-values are taken from this date.  
* `name` - Name of the car.  
* `seller` - Whether the seller is private or a dealer.  
* `offerType` - The type of listing.  
* `price` - The price on the ad to sell the car.  
* `abtest` - Whether the listing is included in an A/B test.  
* `vehicleType` - The vehicle Type.  
* `yearOfRegistration` - The year in which the car was first registered.  
* `gearbox` - The transmission type.  
* `powerPS` - The power of the car in PS.  
* `model` - The car model name.  
* `kilometer` - How many kilometers the car has driven.  
* `monthOfRegistration` - The month in which the car was first registered.  
* `fuelType` - What type of fuel the car uses.  
* `brand` - The brand of the car.  
* `notRepairedDamage` - If the car has a damage which is not yet repaired.  
* `dateCreated` - The date on which the eBay listing was created.  
* `nrOfPictures` - The number of pictures in the ad.  
* `postalCode` - The postal code for the location of the vehicle.  
* `lastSeenOnline` - When the crawler saw this ad last online.

** Let's begin by setting-up the environment.**

In [1]:
# Import Pandas and Numpy
import pandas as pd
import numpy as np

# Read file as pandas dataframe.
autos = pd.read_csv("autos.csv", encoding="Latin-1") # Encoding is Latin-1.

# display the first 5 lines.
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [2]:
# Get detailed info about df's fields (types, counts, nulls e.tc.
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

#### Observations about the dataset.
Most of the field type are objects which means we might need to convert them and we have nulls as well.

In particular, we have 5 out of 20 columns with less than 50.000 non-null values. That means we have to inspect and try to repair those columns.

Those columns are:
* **vehicleType** : 44905 (5095 missing)
* **gearbox** : 47320 (2680 missing)
* **model** : 47242 (2758 missing)
* **fuelType** : 45518 (4482 missing)
* **notRepairedDamage** : 40171 (9829 missing)

**Some things to note:**  
* The "price" along with the "odometer" columns contains data in the U.S. format:  
Comma ( , ) separates thousands.
* The column names use camelcase instead of Python's preferred snakecase, which means we can't just replace spaces with underscores.

## Cleaning Dataset.

** Change the case and the names of columns:**

In [3]:
# Print the column names.
print(autos.columns)

# Convert the column names from camelCase to snake_case.
new_col_names = [
    'date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
    'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
    'odometer', 'registration_month', 'fuel_type', 'brand',
    'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
    'last_seen'
]
autos.columns = new_col_names

# Show the first few lines of the current state of the dataset.
autos.head()

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')


,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Here, we change some things in the column headings of the Dataframe, in order to make it compatible with the Python convention. For example we switch from camelCase to snake_case for the names of the columns and did some other minor changes to the names.

In [4]:
# Get desriptive statistics about both numeric and categorical columns
# of the dataset (include=all)
autos.describe(include="all")

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-08 10:40:35,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


#### Notes on columns
* `price`, `odometer` : Stored as text. They need conversion and column rename.

In [5]:
# Clean the price column.
# --------------------------------------------------------

# Rename the column. 
autos.rename(columns={"price" : "price_$"}, inplace=True) 

# Remove the "$", "," from the value and strip from lead-trail spaces.
autos["price_$"] = autos["price_$"].str.replace("$", "")
autos["price_$"] = autos["price_$"].str.replace(",", "")
autos["price_$"] = autos["price_$"].str.strip()
print("price_$ type before:", autos["price_$"].dtype)

# Convert to float.
autos["price_$"] = autos["price_$"].astype(float)
print("price_$ type after:", autos["price_$"].dtype)

price_$ type before: object
price_$ type after: float64


In [6]:
# Clean the odometer column.
# --------------------------------------------------------------

# Rename the column.
autos.rename(columns={"odometer" : "odometer_km"}, inplace=True)

# Remove the "km", "," from the value and strip from lead-trail spaces.
autos["odometer_km"] = autos["odometer_km"].str.replace("km", "")
autos["odometer_km"] = autos["odometer_km"].str.replace(",", "")
autos["odometer_km"] = autos["odometer_km"].str.strip()
print("odometer_km type before:", autos["odometer_km"].dtype)

autos["odometer_km"] = autos["odometer_km"].astype(int)
print("odometer_km type after:", autos["odometer_km"].dtype)

odometer_km type before: object
odometer_km type after: int64


#### Prices column outliers.
* We can see that in prices column, there are a numerus values of 0, near 0, below 200 and above 350.000. Those are all outliers and needs to be removed.

In [7]:
# Outliers Min and max analysis for price column.
# -----------------------------------------------------------------

# Print all the unique values and show descriptive statistic details.
print("Unique Values:", autos["price_$"].unique().shape, "\n")
print("Describe Details:\n", autos["price_$"].describe(include="all"), "\n")

# Display the frequencies of values in ascending & descending order.
print("Value Counts Ascending:\n", autos["price_$"]
      .value_counts()
      .sort_index(ascending=True)
      .head(), "\n")
print("Value Counts Descending:\n", autos["price_$"]
      .value_counts(10)
      .sort_index(ascending=False)
      .head(), "\n")

Unique Values: (2357,) 

Describe Details:
 count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price_$, dtype: float64 

Value Counts Ascending:
 0.0    1421
1.0     156
2.0       3
3.0       1
5.0       2
Name: price_$, dtype: int64 

Value Counts Descending:
 99999999.0    0.00002
27322222.0    0.00002
12345678.0    0.00006
11111111.0    0.00004
10000000.0    0.00002
Name: price_$, dtype: float64 



In [8]:
# Removing min and max (the outliers) from price column.
# ------------------------------------------------------

# Keep the price range between $200 and $350.000 and remove the rest.
autos = autos[autos["price_$"].between(200, 350000)]
print("Value Counts Descending:\n", autos["price_$"]
      .value_counts(10)
      .sort_index(ascending=False)
      .head(), "\n")

Value Counts Descending:
 350000.0    0.000021
345000.0    0.000021
299000.0    0.000021
295000.0    0.000021
265000.0    0.000021
Name: price_$, dtype: float64 



In [9]:
# Outliers min and max analysis for odometer column.
# ----------------------------------------------------------------
print("Unique Values:", autos["odometer_km"].unique().shape, "\n")
print("Describe Details:\n", autos["odometer_km"].describe(include="all"), "\n")
print("Value Counts Ascending:\n", autos["odometer_km"]
      .value_counts()
      .sort_index(ascending=True)
      .head(), "\n")
print("Value Counts Descending:\n", autos["odometer_km"]
      .value_counts(10)
      .sort_index(ascending=False)
      .head(), "\n")


Unique Values: (13,) 

Describe Details:
 count     47645.000000
mean     125887.501312
std       39482.911790
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64 

Value Counts Ascending:
 5000     720
10000    242
20000    747
30000    773
40000    813
Name: odometer_km, dtype: int64 

Value Counts Descending:
 150000    0.646049
125000    0.104901
100000    0.043719
90000     0.036100
80000     0.029531
Name: odometer_km, dtype: float64 



### Odometer column notes.
* There are only 13 unique values in odometer column and ranges between 5000km and 150000km. Looks like someone had synchronized the odometers for every car.
- Needs further investigation.

## Dates.
Let's now move on to the date columns and understand the date range the data covers.

There are 5 columns that should represent date values. Some of these columns were created by the crawler, some came from the website itself. We can differentiate by referring to the data dictionary:

- `date_crawled`: added by the crawler
- `last_seen`: added by the crawler
- `ad_created`: from the website
- `registration_month`: from the website
- `registration_year`: from the website

In [10]:
# Get the types of the dates columns.
autos[["date_crawled","last_seen", "ad_created", 
       "registration_month", "registration_year"]].dtypes

date_crawled          object
last_seen             object
ad_created            object
registration_month     int64
registration_year      int64
dtype: object

** The dates' columns have problem** : Right now, the **`date_crawled`**, **`last_seen`**, and **`ad_created`** columns are all identified as string values by pandas.

Because these three columns are represented as strings, we need to convert the data into a numerical representation so we can understand it quantitatively.

In [11]:
# Display the format of the problematic dates columns:
autos[['date_crawled','ad_created','last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


In [12]:
# Dates distribution of values.
# -----------------------------------------------------------

# to include missing values in the distribution and to use
# percentages (proportions more accurately) instead of counts
# we use dropna=False & normalize=True.
# Also, get the first 10 chars which contains the date.

# date_crawled
print("date_crawled column: Value Counts Ascending\n", 
      autos["date_crawled"]
      .str[:10] # the first 10 chars
      .value_counts(normalize=True, dropna=False)
      .sort_index(ascending=True)
      .head(), "\n")

# ad_created
print("ad_created column: Value Counts Ascending\n",
      autos["ad_created"]
      .str[:10] # the first 10 chars
      .value_counts(normalize=True, dropna=False)
      .sort_index(ascending=True)
      .head(), "\n")

# last_seen
print("last_seen column: Value Counts Ascending\n",
      autos["last_seen"]
      .str[:10] # the first 10 chars
      .value_counts(normalize=True, dropna=False)
      .sort_index(ascending=True)
      .head(), "\n")


date_crawled column: Value Counts Ascending
 2016-03-05    0.025354
2016-03-06    0.014062
2016-03-07    0.035995
2016-03-08    0.033120
2016-03-09    0.033036
Name: date_crawled, dtype: float64 

ad_created column: Value Counts Ascending
 2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
Name: ad_created, dtype: float64 

last_seen column: Value Counts Ascending
 2016-03-05    0.001091
2016-03-06    0.004303
2016-03-07    0.005373
2016-03-08    0.007178
2016-03-09    0.009613
Name: last_seen, dtype: float64 



In [13]:
# Check the registration year for issues.
print(autos["registration_year"].describe())

count    47645.000000
mean      2004.800084
std         88.423872
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64


#### Registration year
Because the values here begins at year 1000 and ends at year 9999, we have to decide what are the lowest and highest acceptable values:

* The highest year of registration must be not later than 2016 (the year in "last_seen" column.

* The lowest should be no prior to 1920 (and that's for collective cars).

Everything else outside this interval, must be removed.

In [14]:
# Clean the registration_year column.
# -----------------------------------------------------------

# Drop what's fall outside the 1920-2016 interval.
reg_years = autos["registration_year"].between(1920, 2016)
autos = autos[reg_years]

# Calculate the distribution again and sort
# based on the values (the index)
print(autos["registration_year"]
      .value_counts(normalize=True)
      .sort_index(ascending=True)
      .head(10), "\n"
     )
print(autos["registration_year"]
      .value_counts(normalize=True)
      .sort_index(ascending=False)
      .head(10), "\n"
     )

1927    0.000022
1929    0.000022
1931    0.000022
1934    0.000044
1937    0.000087
1938    0.000022
1939    0.000022
1941    0.000044
1943    0.000022
1948    0.000022
Name: registration_year, dtype: float64 

2016    0.025358
2015    0.008212
2014    0.014328
2013    0.017408
2012    0.028547
2011    0.035318
2010    0.034663
2009    0.045409
2008    0.048183
2007    0.049624
Name: registration_year, dtype: float64 



#### Success!
The registration year interval of 1920 - 2016 must be accurate. The first car in the lowest section is registered in 1927. Knowing that no car is registered after 2016, we should be ok.

### The brand column.
When working with data on cars, it's natural to explore variations across different car brands. We can use aggregation to understand the brand column.

In [15]:
# Display the total car brands.
print("Total car brands:", autos["brand"].unique().shape[0])

Total car brands: 40


** The total car brands are 40. We're going to select the top 6 brands based on sales.**

In [16]:
# Collect the car brands.
# -------------------------------------------------------

# Get a list of brands in descending order based on sales
# and get the top 6.
all_brands = autos["brand"].value_counts()
top6_brands = all_brands[:6]

# Create a dict to hold the brands and a counter.
top6_brands_dict = {}
count = 1

# Loop over the top six brands.
for index_brand in top6_brands.index:
#
# Technique A:  with the use of df,
#    total_num_of_cars = top6_brands[index_brand]
#    total_sales_of_cars = autos.loc[
#        autos["brand"] == index_brand, "price_$"].sum()
#     mean_price = total_sales_of_cars / total_num_of_cars
# --------------------------------------------------------------

# Technique B, with "describe()"
    descr = autos.loc[autos["brand"] == index_brand, "price_$"].describe()
    items = int(descr[0])            # [0] == count
    mean_price = round(descr[1], 2)  # [1] == mean
    
    # add brand, mean of sales items to dictionary.
    top6_brands_dict[index_brand] = round(descr[1], 2)

print("")
for key in sorted(top6_brands_dict, key=top6_brands_dict.get, reverse=True):
    print(key, top6_brands_dict[key])


audi 9406.09
mercedes_benz 8691.72
bmw 8402.67
volkswagen 5506.44
ford 3883.29
opel 3078.11


#### ... and, the result says:
**Audi**, the most popular brand!

### Mileage.
For the top 6 brands, let's use aggregation to understand the average mileage for those cars and if there's any visible link with mean price.

We can combine the data from both series objects (popular brand & mileage) into a single dataframe (with a shared index) and display the dataframe directly.

In [17]:
# Create the dataframe using series constructor
# & dataframe constructor.
# ------------------------------------------------------------

# 1. From dictionary to series.
top6_series_const = pd.Series(top6_brands_dict)
print(top6_series_const, "\n")

# 2. From series to dataframe (create a single-column dataframe).
top6_dataframe_const = pd.DataFrame(top6_series_const, columns=["mean_price"])
print(top6_dataframe_const)

audi             9406.09
bmw              8402.67
ford             3883.29
mercedes_benz    8691.72
opel             3078.11
volkswagen       5506.44
dtype: float64 

               mean_price
audi              9406.09
bmw               8402.67
ford              3883.29
mercedes_benz     8691.72
opel              3078.11
volkswagen        5506.44


In [18]:
# Calculate the mean mileage and mean price for each
# of the top brands, storing the results in a dictionary.
# -----------------------------------------------------------------------

# Get the first 6 car brands
all_brands_Final = autos["brand"].value_counts()
top6_brands_Final = all_brands_Final[:6]

# Create a price dict and a mileage dict.
top6_price_dict = {} 
top6_mileage_dict = {}

# Loop through top 6 brands...
for index_brand in top6_brands_Final.index:
    # Get the descriptive statistics
    # of the "price_$" column of the brand.
    des_price = autos.loc[autos["brand"] == index_brand, "price_$"].describe()
    
    # Get the descriptive statistics 
    # of the "odomoter_km" column of the brand.
    des_odom = autos.loc[autos["brand"] == index_brand, "odometer_km"].describe()

    # Add to both dictionaries (price & mileage) the mean prices.
    top6_price_dict[index_brand] = round(des_price[1], 2)
    top6_mileage_dict[index_brand] = round(des_odom[1], 2)
    
print(top6_price_dict)
print(top6_mileage_dict)
 

{'bmw': 8402.67, 'opel': 3078.11, 'volkswagen': 5506.44, 'mercedes_benz': 8691.72, 'audi': 9406.09, 'ford': 3883.29}
{'bmw': 132792.5, 'opel': 129227.52, 'volkswagen': 128774.81, 'mercedes_benz': 131091.72, 'audi': 129260.78, 'ford': 124095.96}


In [19]:
# Convert both dictionaries to series objects, using the series
# constructor. The dictionary key (the brand name) becomes the index.
top6_price_series = pd.Series(top6_price_dict)
top6_mileage_series = pd.Series(top6_mileage_dict)
print("Top 6 brands, based on price:\n", top6_price_series)
print("Mileage of the top 6 brands:\n", top6_mileage_series)

Top 6 brands, based on price:
 audi             9406.09
bmw              8402.67
ford             3883.29
mercedes_benz    8691.72
opel             3078.11
volkswagen       5506.44
dtype: float64
Mileage of the top 6 brands:
 audi             129260.78
bmw              132792.50
ford             124095.96
mercedes_benz    131091.72
opel             129227.52
volkswagen       128774.81
dtype: float64


In [20]:
# Create a dataframe from the first series object
# using the dataframe constructor.
top6_df = pd.DataFrame(top6_price_series, columns=["mean_price"])

# Assign the other series as a new column in this dataframe.
top6_df["mean_mileage"] = top6_mileage_series

top6_df

,mean_price,mean_mileage
audi,9406.09,129260.78
bmw,8402.67,132792.50
ford,3883.29,124095.96
mercedes_benz,8691.72,131091.72
opel,3078.11,129227.52
volkswagen,5506.44,128774.81


### The combination of mean values was successful!
Now, we can easily make any comparison between the top brand based on mean price and the mileage of the cars of that brand.

# The end.